In [1]:
import ipywidgets as widgets
from IPython.display import display, HTML
from collections import Counter

# -----------------------------
# UI Elements
# -----------------------------
number_input = widgets.IntText(
    description="Enter an integer:",
    value=360,
    style={'description_width': 'initial'}
)

display_mode = widgets.Dropdown(
    options=[
        ('List with multiplicities', 'list'),
        ('Unique primes', 'unique'),
        ('Exponent form (canonical)', 'exponent')
    ],
    value='list',
    description='Display:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

ok_button = widgets.Button(
    description="OK",
    button_style='primary',
    tooltip="Factor the number",
)

# Output areas
html_output = widgets.HTML()
error_output = widgets.HTML()

# Quit button
quit_button = widgets.Button(description="Quit")

# Layout
controls = widgets.HBox([display_mode, ok_button])
display(number_input, controls, error_output, html_output, quit_button)


# -----------------------------
# Prime factorization helpers
# -----------------------------
def prime_factors_with_sign(n: int):
    """
    Return (sign, factors) for integer n.
    sign is -1 for negative n, +1 otherwise.
    factors is a list of positive prime factors (with multiplicities).
    Special cases:
      - n == 0: undefined factorization (returns None)
      - |n| == 1: empty factors (units)
    """
    if n == 0:
        return None  # undefined
    sign = -1 if n < 0 else 1
    n = abs(n)

    if n in (0, 1):
        return sign, []

    factors = []

    # Remove factor 2
    while n % 2 == 0:
        factors.append(2)
        n //= 2

    # Remove factor 3
    while n % 3 == 0:
        factors.append(3)
        n //= 3

    # Check 6k ± 1 numbers
    f = 5
    step = 2
    while f * f <= n:
        while n % f == 0:
            factors.append(f)
            n //= f
        f += step
        step = 6 - step  # alternate 2, 4 (i.e., 6k-1, 6k+1)

    if n > 1:
        factors.append(n)

    return sign, factors


# -----------------------------
# Rendering helpers
# -----------------------------
def chips_html(items, colors):
    chip_html = []
    for i, val in enumerate(items):
        color = colors[(i // 10) % len(colors)]
        chip_html.append(
            f"""
            <span style="
                border: 2px solid {color};
                color: {color};
                font-style: italic;
                padding: 4px 8px;
                margin: 2px 4px;
                font-family: monospace;
                display: inline-block;
                border-radius: 8px;
            ">
                {val}
            </span>
            """
        )
    return ''.join(chip_html) if items else '<div style="color:#888;">(none)</div>'


def render_factorization(n: int, sign: int, factors: list[int], mode: str):
    colors = [
        "#FF6B6B", "#4ECDC4", "#45B7D1", "#F9A602",
        "#9B59B6", "#E74C3C", "#3498DB", "#2ECC71",
        "#F1C40F", "#1ABC9C"
    ]

    # Basic info
    abs_n = abs(n)
    is_negative = (sign == -1)
    count = len(factors)
    unique_primes = sorted(set(factors))

    # Title & classification
    if n == 0:
        title = "Prime Factorization of 0 is undefined"
        classification = "0 has infinitely many divisors."
    elif abs_n in (1,):
        title = f"Prime Factorization of {n}"
        classification = "This number is a unit; it has no prime factors."
    elif count == 1 and (factors[0] == abs_n):
        title = f"Prime Factorization of {n}"
        classification = "This number is prime."
    else:
        title = f"Prime Factorization of {n}"
        classification = f"{'Negative; sign -1 × ' if is_negative else ''}Composite with {count} prime factor{'s' if count != 1 else ''}."

    # Build main content per mode
    body_html = ""

    if mode == 'list':
        # Show primes with multiplicities in a chip list
        body_html = chips_html(factors, colors)

    elif mode == 'unique':
        # Show unique primes only
        body_html = chips_html(unique_primes, colors)

    elif mode == 'exponent':
        # Show canonical exponent form: n = (-1) × ∏ p^k
        if n == 0:
            body_html = '<div style="color:#888;">(undefined)</div>'
        else:
            cnt = Counter(factors)
            parts = []
            for p in sorted(cnt.keys()):
                k = cnt[p]
                if k == 1:
                    parts.append(f"{p}")
                else:
                    parts.append(f"{p}<sup>{k}</sup>")
            product = " × ".join(parts) if parts else "1"
            sign_str = "-1 × " if is_negative else ""
            body_html = (
                f"<div style='font-family: monospace; font-size: 18px;'>"
                f"{n} = {sign_str}{product}"
                f"</div>"
            )

    # Wrap in card
    html_content = f"""
    <div style="font-family: system-ui, -apple-system, Segoe UI, Roboto, Arial; max-width: 900px;">
        <div style="text-align: center; margin-bottom: 10px;">
            <h3 style="font-weight: 700; margin: 0;">{title}</h3>
            <div style="color:#444; font-size: 13px; margin-top: 6px;">
                {classification}
            </div>
        </div>
        <div style="
            border: 1px solid #ddd; border-radius: 10px; padding: 10px;
            max-height: 320px; overflow: auto; background: #fafafa;
        ">
            {body_html}
        </div>
        <div style="color:#888; font-size: 12px; margin-top: 8px;">
            Note: Trial division (2, 3, then 6k±1). Very large integers may take a long time.
        </div>
    </div>
    """
    return html_content


# -----------------------------
# Event handlers
# -----------------------------
def on_ok_clicked(b):
    error_output.value = ""  # clear previous errors
    try:
        n = number_input.value
        mode = display_mode.value

        # Validate
        if n is None or not isinstance(n, int):
            error_output.value = "<div style='color:red;'>Please enter a valid integer.</div>"
            html_output.value = ""
            return

        # Edge case: 0 (undefined factorization)
        if n == 0:
            html_output.value = render_factorization(n, 1, [], mode)
            return

        # Factor
        result = prime_factors_with_sign(n)
        if result is None:
            # Shouldn't reach here because we handled 0 above
            error_output.value = "<div style='color:red;'>Factorization is undefined for 0.</div>"
            html_output.value = ""
            return

        sign, factors = result
        html_output.value = render_factorization(n, sign, factors, mode)

    except Exception as err:
        error_output.value = f"<div style='color:red;'>An error occurred: {err}</div>"
        html_output.value = ""


ok_button.on_click(on_ok_clicked)


def on_quit_button_clicked(b):
    # Clear output and close the widgets
    html_output.value = ""
    error_output.value = ""
    number_input.close()
    display_mode.close()
    ok_button.close()
    quit_button.close()
    html_output.close()
    error_output.close()

quit_button.on_click(on_quit_button_clicked)

# Optional: auto-run once on display
# on_ok_clicked(None)

IntText(value=360, description='Enter an integer:', style=DescriptionStyle(description_width='initial'))

HTML(value='')

HTML(value='')

Button(description='Quit', style=ButtonStyle())